In [1]:
#!pip install --upgrade yahoo_fin

In [2]:
import pandas as pd
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from yahoo_fin.stock_info import get_data
from flask import Blueprint
import requests
import json
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine
from flask import Flask, render_template
from os import path
import csv
import pprint

import pandas as pd
Finance_db = "database.db" ## defining the name of the SQLite database
db = SQLAlchemy()



In [3]:
## Import NYSE stock from CSV
NYSE_stock_df = pd.read_csv('/Users/ellandalla/Desktop/website Project/Data/NYSE_stocks.csv')

In [4]:
NYSE_stock_df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$125.05,-2.66,-2.08%,3.693674e+10,United States,1999.0,1742578,Industrials,Biotechnology: Laboratory Analytical Instruments
1,AA,Alcoa Corporation Common Stock,$34.50,0.57,1.68%,6.156511e+09,United States,2016.0,4243329,Industrials,Aluminum
2,AAIC,Arlington Asset Investment Corp Class A (new),$4.43,-0.12,-2.64%,1.256368e+08,United States,NaN,289165,Real Estate,Real Estate Investment Trusts
3,AAIN,Arlington Asset Investment Corp 6.000% Senior ...,$23.71,0.00,0.00%,6.724234e+08,United States,NaN,10,Real Estate,Real Estate Investment Trusts
4,AAN,Aarons Holdings Company Inc. Common Stock,$13.15,-1.08,-7.59%,4.059274e+08,United States,2020.0,294314,Consumer Discretionary,Diversified Commercial Services


In [5]:
list = NYSE_stock_df['Sector'].unique().tolist()
print(list)

['Industrials', 'Real Estate', 'Consumer Discretionary', 'Finance', 'Health Care', 'Consumer Staples', 'Utilities', 'Technology', 'Energy', 'Telecommunications', 'Basic Materials']


In [6]:
stock_df_results = pd.DataFrame()
# Use a for loop to fetch historical data from YFinance
for symbol in NYSE_stock_df['Symbol']:
    try:
        data = get_data(symbol, start_date="2014-01-01", end_date="2023-07-31", index_as_date = True, interval="1mo")
        if not data.empty:
            stock_df_results = pd.concat([stock_df_results, data])
        else:
            print(f"No data available for {symbol}")
        # Process the data here
    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        
      

Error fetching data for ABC: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error fetching data for APE: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error fetching data for ARNC: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error fetching data for CEN: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error fetching data for DMS: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error fetching data for ECC           : {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error fetching data for HHC: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, 

In [7]:
stock_df_results.iloc[:5]

,open,high,low,close,adjclose,volume,ticker
2014-01-01,40.844063,43.791130,40.164520,41.595135,38.273087,73379205.0,A
2014-02-01,41.595135,43.247498,39.248928,40.722462,37.470104,99904852.0,A
2014-03-01,40.143063,42.432045,38.783978,40.000000,36.805359,71466178.0,A
2014-04-01,40.200287,40.944206,37.167381,38.655224,35.567982,59032368.0,A
2014-05-01,38.583691,41.044350,38.383404,40.729614,37.567413,64035252.0,A


In [8]:
stock_df_results.iloc[:-5]

,open,high,low,close,adjclose,volume,ticker
2014-01-01,40.844063,43.791130,40.164520,41.595135,38.273087,73379205.0,A
2014-02-01,41.595135,43.247498,39.248928,40.722462,37.470104,99904852.0,A
2014-03-01,40.143063,42.432045,38.783978,40.000000,36.805359,71466178.0,A
2014-04-01,40.200287,40.944206,37.167381,38.655224,35.567982,59032368.0,A
2014-05-01,38.583691,41.044350,38.383404,40.729614,37.567413,64035252.0,A
...,...,...,...,...,...,...,...
2022-10-01,24.809999,26.040001,21.389999,23.490000,23.159281,32707400.0,ZWS
2022-11-01,23.750000,25.340000,21.910000,24.209999,23.869141,28095300.0,ZWS
2022-12-01,24.420000,25.559999,20.650000,21.150000,20.912745,20613500.0,ZWS
2023-01-01,21.450001,23.160000,20.100000,21.860001,21.614779,34046400.0,ZWS


In [9]:
# merge the stock_df_results with stock_df to get the sector
NYSE_final_df = pd.merge(NYSE_stock_df[['Symbol', 'Sector']], stock_df_results.reset_index(), 
                            left_on="Symbol", right_on="ticker")
## Group stock by sector and by price
grouped = NYSE_final_df.groupby(['Sector', 'index']).sum()
grouped_data = pd.DataFrame(grouped)

#index_values = grouped_data.index.get_level_values('index')
#print(index_values)
#grouped_data


drop_list=['high', 'low', 'adjclose', 'open' ]
for item in drop_list:
    grouped_data = grouped_data.drop(item, axis=1)
grouped_data.tail()

/var/folders/73/hc_zb17944z42_dsmk15lr780000gn/T/ipykernel_50540/1814545652.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = NYSE_final_df.groupby(['Sector', 'index']).sum()


close        volume
Sector    index                                
Utilities 2023-03-01  4799.160012  4.186313e+09
          2023-04-01  4827.734996  2.764369e+09
          2023-05-01  4619.944992  3.801694e+09
          2023-06-01  4658.304979  3.741476e+09
          2023-07-01  4730.679994  3.190858e+09

In [10]:


grouped_data.head()

close        volume
Sector          index                               
Basic Materials 2014-01-01  358.965408  1.084709e+09
                2014-02-01  376.895637  1.003728e+09
                2014-03-01  361.012109  9.948548e+08
                2014-04-01  361.811204  9.037479e+08
                2014-05-01  344.768924  7.695150e+08

In [11]:
#grouped_data.reset_index(inplace=True)


In [12]:
## calculate % change 
grouped_data['Change in Price'] = grouped_data.groupby('Sector')['close'].pct_change() * 100
grouped_data['Volume_Percentage_Change']= grouped_data.groupby('Sector')['volume'].pct_change() * 100
grouped_data.tail()

close        volume  Change in Price  \
Sector    index                                                    
Utilities 2023-03-01  4799.160012  4.186313e+09         2.032503   
          2023-04-01  4827.734996  2.764369e+09         0.595416   
          2023-05-01  4619.944992  3.801694e+09        -4.304089   
          2023-06-01  4658.304979  3.741476e+09         0.830313   
          2023-07-01  4730.679994  3.190858e+09         1.553677   

                      Volume_Percentage_Change  
Sector    index                                 
Utilities 2023-03-01                 31.131175  
          2023-04-01                -33.966512  
          2023-05-01                 37.524844  
          2023-06-01                 -1.583983  
          2023-07-01                -14.716594

In [13]:
pivot_df_Price = pd.pivot_table(grouped_data, index = ['index'], columns = ['Sector'], values = 'Change in Price').reset_index()

In [14]:
pivot_df_Close = pd.pivot_table(grouped_data, index = ['index'], columns = ['Sector'], values = 'close').reset_index()
pivot_df_Close.tail()

Sector,index,Basic Materials,Consumer Discretionary,Consumer Staples,Energy,Finance,Health Care,Industrials,Real Estate,Technology,Telecommunications,Utilities
110,2023-03-01,518.539996,31886.253059,2575.203243,5192.710030,18155.625950,10138.216188,21478.440955,5873.598905,7914.106603,1119.229995,4799.160012
111,2023-04-01,517.560006,32597.759354,2594.721736,4946.770015,18500.667023,10636.047958,21058.481803,5806.185921,7549.314647,1164.699971,4827.734996
112,2023-05-01,461.919992,31355.126047,2500.244843,4516.959949,17819.227917,10291.925879,20099.216758,5572.646803,8000.574487,986.590018,4619.944992
113,2023-06-01,522.099998,34743.313673,2496.416382,4819.160007,18895.931851,11024.858242,22525.015735,5878.116460,8465.985657,1055.410012,4658.304979
114,2023-07-01,569.779997,35563.133003,2616.367495,5347.630065,19835.175012,10932.149983,23206.151896,6029.451045,8839.219958,1106.340002,4730.679994


In [15]:
pivot_df_Price.tail()

Sector,index,Basic Materials,Consumer Discretionary,Consumer Staples,Energy,Finance,Health Care,Industrials,Real Estate,Technology,Telecommunications,Utilities
109,2023-03-01,-2.588670,0.441780,2.276523,-3.402044,-7.075990,-0.525517,-1.360628,-5.759037,3.854676,0.680964,2.032503
110,2023-04-01,-0.188990,2.231389,0.757940,-4.736256,1.900464,4.910447,-1.955259,-1.147729,-4.609389,4.062612,0.595416
111,2023-05-01,-10.750447,-3.812021,-3.641119,-8.688701,-3.683322,-3.235432,-4.555243,-4.022247,5.977494,-15.292346,-4.304089
112,2023-06-01,13.028232,10.805849,-0.153123,6.690342,6.042371,7.121431,12.069122,5.481590,5.817222,6.975541,0.830313
113,2023-07-01,9.132350,2.359646,4.804932,10.966020,4.970610,-0.840902,3.023910,2.574542,4.408634,4.825612,1.553677


In [16]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///finance.db')
pivot_df_Price.to_sql("Cagr_Data", engine, index=False, if_exists="replace")

114

In [17]:
CAGR_data = {}
for sector in pivot_df_Price.columns[1:]:
    # initiaitlize the sector names in dictionary
    CAGR_data[sector] = {}
    # Add the keys in the deictionary for the sector key 
    CAGR_data[sector]['X'] = pivot_df_Price['index'].astype(str).to_list()
    CAGR_data[sector]['Y'] = pivot_df_Price[sector].to_list()
print(type(CAGR_data))

<class 'dict'>


In [18]:
## converting to json file
file_path = "CAGR_data.json"
with open(file_path, "w") as json_file:
    json.dump(CAGR_data, json_file)

In [19]:
Sector_data = {}
for sector in pivot_df_Close.columns[1:]:
    Sector_data[sector] = {}
    Sector_data [sector]['Y'] = pivot_df_Close[sector].to_list()
    
    

In [20]:
## converting to json file
file_path = "Sector_data.json"
with open(file_path, "w") as json_file:
    json.dump(Sector_data, json_file )

In [21]:
# To capture CAGR Volume 
pivot_df_Volume = pd.pivot_table(grouped_data, index = ['index'], columns = ['Sector'], values = 'Volume_Percentage_Change').reset_index()

In [22]:

pivot_df_Volume

Sector,index,Basic Materials,Consumer Discretionary,Consumer Staples,Energy,Finance,Health Care,Industrials,Real Estate,Technology,Telecommunications,Utilities
0,2014-02-01,-7.465700,-12.312843,7.182717,8.900178,-18.298079,-10.486702,-6.803096,-5.039706,-16.260215,-9.014299,-6.568503
1,2014-03-01,-0.883979,-2.807732,-1.094541,-4.575313,14.310027,4.290445,-5.540875,-1.837079,21.575483,3.834304,3.008165
2,2014-04-01,-9.157813,6.247666,-4.913327,0.785023,-1.413062,3.054655,-0.681329,-1.102804,2.322361,-4.997497,2.429471
3,2014-05-01,-14.852910,-14.018700,-20.947077,-13.550678,-19.318753,-13.221722,-11.940087,-1.362390,-17.624377,-9.843298,-5.007986
4,2014-06-01,4.139509,2.059945,18.714764,6.430236,-5.191523,-1.599614,-1.263888,6.813335,-10.240796,-23.757644,2.178008
...,...,...,...,...,...,...,...,...,...,...,...,...
109,2023-03-01,38.249167,25.107542,37.462259,22.314672,130.379133,25.040936,27.438798,65.654991,7.207923,21.753662,31.131175
110,2023-04-01,-26.549509,-33.794016,-36.161909,-35.896577,-44.411013,-25.705879,-24.874283,-38.310380,-33.065408,-29.575202,-33.966512
111,2023-05-01,14.597670,39.569737,38.726134,28.210379,20.624525,25.909465,22.848852,22.677078,65.324472,15.942274,37.524844
112,2023-06-01,-6.271671,11.116540,0.446089,-3.430391,-15.883236,-6.045700,7.119015,7.979132,4.407743,9.707089,-1.583983


In [26]:
pivot_df_Volume.to_sql("Cagr_Volume", engine, index=False, if_exists="replace")

114

In [23]:
CAGR_Volume = {}
for sector in pivot_df_Volume.columns[1:]:
    # initiaitlize the sector names in dictionary
    CAGR_Volume[sector] = {}
    # Add the keys in the deictionary for the sector key 
    CAGR_Volume[sector]['X'] = pivot_df_Volume['index'].astype(str).to_list()
    CAGR_Volume[sector]['Y'] = pivot_df_Volume[sector].to_list()

In [24]:
file_path = "CAGR_volume.json"
with open(file_path, "w") as json_file:
    json.dump(CAGR_Volume, json_file)

In [25]:
## compounding  all jsons into 1 file


json_lst = ['CAGR_volume.json', 'Sector_data.json', 'CAGR_data.json', 'CPI_Data.json', 'Description.json' ]
merged_data = {}
for item in json_lst:
    with open(item, 'r') as file:
        data = json.load(file)
        merged_data.update(data)
# Write the merged data to a new file
with open('merged.json', 'w') as merged_file:
    json.dump(merged_data, merged_file, indent=2)
        
